### while else clause

only useful when break clause is present

In [ ]:
if condition:
    execute_condition_is_true()
else:
    execute_condition_is_false()

In [ ]:
while condition:
    execute_condition_is_true()
else:
    execute_condition_is_false()
    
# which would be equivalent to
while condition:
    execute_condition_is_true()
execute_condition_is_false()

In [ ]:
while condition:
    flag = execute_condition_is_true()
    if flag:
        break
        
execute_condition_is_false() # which will be executed whether condition is True or False

In [ ]:
# a fix but with redundant test
while condition:
    flag = execute_condition_is_true()
    if flag:
        break
        
if not condition:        
    execute_condition_is_false()

In [ ]:
# a fix without redundant test
while condition:
    flag = execute_condition_is_true()
    if flag:
        break        
else: # no break       
    execute_condition_is_false()

In [1]:
import operator

In [2]:
def is_comment(item):
    return isinstance(item, str) and item.startswith('#')

In [3]:
def execute(program):
    while program:
        item = program.pop()
        if not is_comment(item):
            program.append(item)
            break
    else: # no break
        print("Empty program")
        return

    pending = []
    while program:
        item = program.pop()
        if callable(item):
            try:
                result = item(*pending)
            except Exception as e:
                print('error:', e)
                break
                
            program.append(result)
            pending.clear()
        else:
            pending.append(item)
            
    else:
        print('Program successful!')
        print('Result:', pending)
    
    print('Finished!')

In [4]:
program = list(reversed(('# A short stack program to add', 
                         '# and multiply some constants',
                         5,
                         2,
                         operator.add,
                         3,
                         operator.mul
                        )))

execute(program)

Program successful!
Result: [21]
Finished!


### for else clause

- mostly useful in a searching algorithm
- else clause useful as not-found clause

In [ ]:
for item in iterable:
    if match(item):
        result = item
        break
else: # no break
    # no match found
    result = None
    
# always come here
print(result)

In [5]:
items = [2, 36, 25, 9]
divisor = 12

for item in items:
    if item % divisor == 0:
        found = item
        break
else:
    items.append(divisor)
    found = divisor
    
print("{items} contains {found}, which is a multiple of {divisor}".format(**locals()))

[2, 36, 25, 9] contains 36, which is a multiple of 12


### try else clause

- else clause is a success clause
- narrow down the scope of try block
- clarify about from where exceptions are raised

In [ ]:
try:
    do_something()
except ValueError:
    # ValueError caught and handled
    handle_value_error()
else:
    # no exception was raised
    # we know do_something succeeded
    do_something_else()

In [ ]:
try:
    do_something()
    do_something_else()
except ValueError: # which caused ValueError? Ambiguous!!!
    # ValueError caught and handled
    handle_value_error()

In [ ]:
try:
    f = open(filename, 'r')
except OSError:
    print('File could not be opened')
else:
    # now we are sure the file is open
    print("# of lines:", sum(1 for line in f))
    f.close()

### Emulating switch statements

The switch statement in C

switch (menu_option) {\
    case 1: single_player(); break;\
    case 2: multi_player(); break;\
    case 3: load_game(); break;\
    case 4: save_game(); break\
    case 5: reset_high_score(); break;\
    default:\
        printf('No such option');\
        break;\
}

There is no switch construction in Python.
1. Option 1: if ... elif ... elif ... else; tedious to write and error prone
2. Option 2: mapping of callables

In [6]:
def play():
    
    position = (0, 0)
    
    while position:
        if position == (0, 0):
            print('You are in a maze of twisted passages, all alike')
        elif position == (1, 0):
            print('You are on a road in a dark forest. To the north you can see a tower')
        elif position == (1, 1):
            print('There is a tall tower there, with no obvious door. A path leads east')
        else:
            print('There is nothing here')

In [8]:
def play():
    
    position = (0, 0)

    locations = {
    (0, 0): lambda: print('You are in a maze of twisted passages, all alike'),
    (1, 0): lambda: print('You are on a road in a dark forest. To the north you can see a tower'),
    (1, 1): lambda: print('There is a tall tower there, with no obvious door. A path leads east'),
}
    
    try:
        location_action = locations[position]
    except KeyError:
        print('There is nothing here')
    else:
        location_action()

### Dispatching on type

- functions selected based on type of arguments
- methods: called implementation based on type of self
- regular functions: switch-emulation is ungainly
- use the @singledispatch decorator

Functions which support multiple implementation depending on the type of arguments are called generic functions. And each version of generic functions is referred to as an overload of the function. The act of providing another version of generic function for different argument types is called overloading the function. 

Do not use single dispatch on methods as single dispatch is based only on the type of the first argument and self is actually the first argument. The solution is to move the generic function out of the class.

Generic functions are functions defined for polymorphism.

In [9]:
from functools import singledispatch

@singledispatch
def fun(arg, verbose=False): # generic function
    if verbose:
        print("Let me just say,", end=' ')
    print(arg)
    
@fun.register(int)
def _(arg, verbose=False):
    if verbose:
        print("Strength in numbers")
    print(arg)
    
@fun.register(list)
def _(arg, verbose=False):
    if verbose:
        print("Enumerate this!")
    for i, elem in enumerate(arg):
        print(i, elem)

In [12]:
@singledispatch
def draw(shape):
    print("Dont know how to draw {!r}".format(shape))
    
@draw.register(Circle)
def _(shape):
    print("Draw a ", 'circle')
    
@draw.register(Parallelogram)
def _(shape):
    print("Draw a ", 'parallelogram')

@draw.register(Triangle)
def _(shape):
    print("xxxx")

NameError: name 'Circle' is not defined